# 🎨 FONTe AI - Font Generation Training

Train an AI model to generate unique fonts using SVG path transformers.

**Requirements:**
- Google Colab (Free tier works!)
- T4 GPU (16GB) - plenty for this model
- ~6-10 hours for full training

**What this notebook does:**
1. Setup environment
2. Upload/download training data
3. Train the model
4. Generate sample fonts
5. Export model

## 1️⃣ Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Mount Google Drive (for saving checkpoints)
from google.colab import drive
drive.mount('/content/drive')

# Create project directory
!mkdir -p /content/fonte_ai
%cd /content/fonte_ai

## 2️⃣ Upload Training Data

Upload your `TOKENIZED` folder from local machine, or clone from GitHub.

In [ ]:
# Option A: Upload from local (run this cell, then drag & drop files)
from google.colab import files

# Upload train.bin, val.bin, vocabulary.json
print("Upload your training files (train.bin, val.bin, vocabulary.json):")
uploaded = files.upload()

# Move to proper location
!mkdir -p TOKENIZED
!mv train.bin val.bin vocabulary.json TOKENIZED/ 2>/dev/null || true
!ls -la TOKENIZED/

In [ ]:
# Option B: Copy from Google Drive
# (If you've uploaded the TOKENIZED folder to Drive)
# !cp -r /content/drive/MyDrive/fonte_ai/TOKENIZED .

## 3️⃣ Model Definition

In [ ]:
import math
import json
import struct
from pathlib import Path
from typing import Optional, Dict, List, Tuple
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR


@dataclass
class ModelConfig:
    vocab_size: int = 1105
    max_seq_length: int = 512
    d_model: int = 256
    n_heads: int = 4
    n_layers: int = 6
    d_ff: int = 1024
    dropout: float = 0.1
    pad_token_id: int = 0
    sos_token_id: int = 1
    eos_token_id: int = 2


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 512, dropout: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.shape
        q = self.w_q(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        k = self.w_k(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        v = self.w_v(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        return self.w_o(out)


class FeedForward(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.linear2(self.dropout(F.gelu(self.linear1(x))))


class TransformerBlock(nn.Module):
    def __init__(self, d_model: int, n_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, n_heads, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x = x + self.dropout(self.attention(self.norm1(x), mask))
        x = x + self.dropout(self.ff(self.norm2(x)))
        return x


class FonteModel(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config
        self.token_embedding = nn.Embedding(config.vocab_size, config.d_model, padding_idx=config.pad_token_id)
        self.pos_encoding = PositionalEncoding(config.d_model, config.max_seq_length, config.dropout)
        self.blocks = nn.ModuleList([
            TransformerBlock(config.d_model, config.n_heads, config.d_ff, config.dropout)
            for _ in range(config.n_layers)
        ])
        self.norm = nn.LayerNorm(config.d_model)
        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.lm_head.weight = self.token_embedding.weight
        self.apply(self._init_weights)
        self.register_buffer('causal_mask', torch.tril(torch.ones(config.max_seq_length, config.max_seq_length)))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, input_ids, labels=None):
        batch_size, seq_len = input_ids.shape
        x = self.token_embedding(input_ids)
        x = self.pos_encoding(x)
        mask = self.causal_mask[:seq_len, :seq_len]
        for block in self.blocks:
            x = block(x, mask)
        x = self.norm(x)
        logits = self.lm_head(x)
        result = {'logits': logits}
        if labels is not None:
            shift_logits = logits[:, :-1, :].contiguous()
            shift_labels = labels[:, 1:].contiguous()
            loss = F.cross_entropy(shift_logits.view(-1, self.config.vocab_size), shift_labels.view(-1), ignore_index=self.config.pad_token_id)
            result['loss'] = loss
        return result

    @torch.no_grad()
    def generate(self, style_id, char_id, max_length=256, temperature=1.0, top_k=50, top_p=0.9):
        self.eval()
        device = next(self.parameters()).device
        tokens = torch.tensor([[self.config.sos_token_id, style_id, char_id]], device=device)
        for _ in range(max_length - 3):
            outputs = self.forward(tokens)
            logits = outputs['logits'][:, -1, :] / temperature
            if top_k > 0:
                indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
                logits[indices_to_remove] = float('-inf')
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            tokens = torch.cat([tokens, next_token], dim=1)
            if next_token.item() == self.config.eos_token_id:
                break
        return tokens

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def save(self, path):
        torch.save({'config': self.config.__dict__, 'state_dict': self.state_dict()}, path)

    @classmethod
    def load(cls, path, device='cpu'):
        checkpoint = torch.load(path, map_location=device)
        config = ModelConfig(**checkpoint['config'])
        model = cls(config)
        model.load_state_dict(checkpoint['state_dict'])
        return model


print("✅ Model classes defined!")

## 4️⃣ Dataset Loading

In [ ]:
class FonteDataset(Dataset):
    def __init__(self, data_path, max_length=512):
        self.max_length = max_length
        with open(data_path, 'rb') as f:
            num_sequences, max_length, vocab_size = struct.unpack('III', f.read(12))
            self.token_ids = []
            self.lengths = []
            for _ in range(num_sequences):
                length = struct.unpack('H', f.read(2))[0]
                tokens = list(struct.unpack(f'{max_length}H', f.read(max_length * 2)))
                self.lengths.append(length)
                self.token_ids.append(tokens)
        print(f"Loaded {len(self.token_ids)} sequences")

    def __len__(self):
        return len(self.token_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.token_ids[idx], dtype=torch.long),
            'length': self.lengths[idx],
        }


# Load datasets
train_dataset = FonteDataset('TOKENIZED/train.bin')
val_dataset = FonteDataset('TOKENIZED/val.bin')

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## 5️⃣ Training Configuration

In [ ]:
# Training hyperparameters
BATCH_SIZE = 64          # Increase for faster training on T4
EPOCHS = 50              # 50-100 for good results
LEARNING_RATE = 3e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model config (medium size ~12M params)
config = ModelConfig(
    vocab_size=1105,
    max_seq_length=512,
    d_model=256,
    n_heads=4,
    n_layers=6,
    d_ff=1024,
    dropout=0.1,
)

# Create model
model = FonteModel(config).to(DEVICE)
print(f"Model parameters: {model.count_parameters():,}")
print(f"Device: {DEVICE}")

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=len(train_loader) * EPOCHS)

## 6️⃣ Training Loop

In [ ]:
import time
from tqdm.auto import tqdm

def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc="Training")
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        outputs = model(input_ids, labels=input_ids)
        loss = outputs['loss']
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
    return total_loss / len(dataloader)

@torch.no_grad()
def validate(model, dataloader, device):
    model.eval()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        outputs = model(input_ids, labels=input_ids)
        total_loss += outputs['loss'].item()
    return total_loss / len(dataloader)

# Training
best_val_loss = float('inf')
history = []

print("🚀 Starting training...")
start_time = time.time()

for epoch in range(1, EPOCHS + 1):
    epoch_start = time.time()
    
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, DEVICE)
    val_loss = validate(model, val_loader, DEVICE)
    
    epoch_time = time.time() - epoch_start
    history.append({'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss})
    
    print(f"Epoch {epoch}/{EPOCHS} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | Time: {epoch_time:.1f}s")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save('best_model.pt')
        print(f"  💾 Saved best model!")
    
    # Save to Drive every 10 epochs
    if epoch % 10 == 0:
        model.save(f'/content/drive/MyDrive/fonte_ai/checkpoint_epoch_{epoch}.pt')

total_time = time.time() - start_time
print(f"\n✅ Training complete in {total_time/60:.1f} minutes!")
print(f"Best validation loss: {best_val_loss:.4f}")

## 7️⃣ Test Generation

In [ ]:
# Load best model
model = FonteModel.load('best_model.pt', device=DEVICE)
model = model.to(DEVICE)

# Style and character token mappings
STYLE_IDS = {
    'serif': 28,
    'sans-serif': 29,
    'monospace': 30,
    'handwriting': 31,
    'display': 32,
}

CHAR_IDS = {char: 33 + i for i, char in enumerate("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789!@#$%&*()-+=[]")}

# Generate a sample
style = 'serif'
char = 'A'

tokens = model.generate(
    style_id=STYLE_IDS[style],
    char_id=CHAR_IDS[char],
    max_length=256,
    temperature=0.8,
    top_k=50,
)

print(f"Generated {style} '{char}':")
print(f"Tokens: {tokens[0].tolist()[:30]}...")
print(f"Total tokens: {tokens.shape[1]}")

## 8️⃣ Save Final Model

In [ ]:
# Save to Drive
!mkdir -p /content/drive/MyDrive/fonte_ai

model.save('/content/drive/MyDrive/fonte_ai/final_model.pt')
print("✅ Model saved to Google Drive!")

# Save training history
with open('/content/drive/MyDrive/fonte_ai/training_history.json', 'w') as f:
    json.dump(history, f, indent=2)

# Download to local
from google.colab import files
files.download('best_model.pt')

## 🎉 Done!

Your model is trained! Next steps:
1. Download `best_model.pt`
2. Use the generation script to create fonts
3. Convert to TTF using svgtofont